<h1>Capstone Project - Battle of Neighborhoods (week 2)</h1>
<h2>Applied Data Science Capstone</h2>

<h3>Introduction to Business Problem</h3>

<h3>Data</h3>

<h3>Getting the location</h3>

In [6]:
#Installing Required modules
#!conda install -c anaconda geocoder -y
#!pip install geocoder

In [7]:
#Importing required libraries
import numpy as npu
import pandas as pd

from geopy.geocoders import Nominatim
import geocoder

import requests
from bs4 import BeautifulSoup

In [5]:
g = geocoder.arcgis('Bangalore, India')
gll = g.latlng
print(gll)

[12.966870000000029, 77.58734000000004]


<h3>Getting the List of Neighborhoods in Bangalore from Wikipedia</h3>

In [8]:
neig = requests.get("https://commons.wikimedia.org/wiki/Category:Suburbs_of_Bangalore").text

In [35]:
soup = BeautifulSoup(neig, 'html.parser')

In [79]:
#Creating a list to store neighborhood data
neighborhoodlist = []

In [80]:
for i in soup.find_all('div', class_='mw-category')[0].find_all('a'):
    neighborhoodlist.append(i.text)
#Creating a dataframe from the list
neig_df = pd.DataFrame({"Localities": neighborhoodlist})
neig_df.head()

,Localities
0,"Agara, Bangalore"
1,Arekere
2,Banashankari
3,Banaswadi
4,Basavanagudi


In [81]:
neig_df.shape

(58, 1)

<h3>Getting the location of the Localities</h3>

In [108]:
#Defining a function to get the location of the localities
def get_location(localities):
    g = geocoder.arcgis('{}'.format(localities))
    get_latlng = g.latlng
    return get_latlng

In [124]:
co_ordinates = []
for i in neig_df["Localities"].tolist():
    co_ordinates.append(get_location(i))
print(co_ordinates)

[[12.842700000000036, 77.48882000000003], [12.885640000000024, 77.59669000000008], [12.922280000000057, 77.56986000000006], [13.015760000000057, 77.64871000000005], [12.939000000000021, 77.57135000000005], [12.882490000000075, 77.62475000000006], [12.927350000000047, 77.67184000000003], [23.37385000000006, 85.35071000000005], [12.966870000000029, 77.58734000000004], [12.798850000000073, 77.38980000000004], [12.817540000000065, 77.67879000000005], [12.966210117271197, 77.60678848215437], [12.793990000000065, 77.70018000000005], [13.250110000000063, 77.70788000000005], [26.873360000000048, 80.91627000000005], [12.943300000000022, 77.65603000000004], [12.845780000000047, 77.67274000000003], [12.998850000000061, 77.61271000000005], [23.633330000000058, 90.25000000000006], [13.02642000000003, 77.62432000000007], [13.049690000000055, 77.58951000000008], [13.077180000000055, 77.80178000000006], [12.912160000000029, 77.64490000000006], [12.995470000000068, 80.25227000000007], [12.9234400000000

In [125]:
#Creating a dataframe from the list of location
co_ordinates_df = pd.DataFrame(co_ordinates, columns=['Latitudes', 'Longitudes'])

In [ ]:
neig_df["Latitudes"] = co_ordinates_df["Latitudes"]
neig_df["Lon"] = co_ordinates_df["Longitudes"]